# Demographics& partial EDA CW1 

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_theme(style="whitegrid")
sns.set_palette('Blues_r')

pd.set_option('display.max_rows', None) # Displays all rows in the table
pd.set_option('display.max_columns', None) # Displays all columns in the table

# Define decimal places shown
pd.options.display.float_format = '{:,.3f}'.format

We load the dataset, which is cleaned of approximately 40k duplicates:

In [2]:
df = pd.read_csv('00_data/clean_duplicates.csv', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '00_data/clean_duplicates.csv'

In [ ]:
#pd.set_option('display.max_rows', None) # Displays all rows in the table
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

#### Get General overview of complete dataframe

In [ ]:
df.shape

In [ ]:
df.describe().T

#### Check max values for:
- cnt_abo(7,104) 
- cnt_abo_diezeit(3,708)
- cnt_abo_diezeit_digital (516)
- cnt_abo_magazin(414)
- cnt_umwandlungsstatus2_dkey(3,129)

In [ ]:
df.query("cnt_abo > 500").sort_values('cnt_abo', ascending=False).head(5)

In [ ]:
df_zeitbrief = df.drop(df.iloc[:, 22:143], axis=1)
df_zeitbrief.columns

In [ ]:
df_zeitbrief.head()

In [ ]:
df_zeitbrief.describe().T

In [ ]:
df_zeitbrief.info()

In [ ]:
df_zeitbrief.nunique()

#### Conclusions on Subset:

- there probably are outliers regarding "received_anzahl_zeitbrief" & "shop_kauf and the click- & openrates"
- some of the zeitbrief clickrates & openrates have values > 1, we should check after removing the outliers

# Outliers:

#### Shop-Kauf
    - 87.7% have never ordered in the zeit-shop
    - 99% of the customers order less than 10 Products from the zeit shop 
    - only 3 customers order 152 products, the runners up order 80 Units from the zeitshop, from 80 orders down the dirstribution is continouus, so we delete the 3 Outliers. 

In [ ]:
df_zeitbrief.query('shop_kauf > 1').sort_values('shop_kauf', ascending=False).head(1)

In [ ]:
df_zeitbrief.shop_kauf.value_counts(1).head(5)

In [ ]:
df_outlier = df[df.shop_kauf < 100]

#### Zeitbrief:

- >99% of the subscribers received (and opened) less than 29 newsletters in the last 6m 
- 56 subscribers received over 29 newsletters in the last 6m
    - they received 4 newsletters per week (wednesdays), that adds up to 100 - 108 in the last 6m (depends on the day you subscribe)
    - they have the same nl - registration dates (2018-02-18) and earliest delivery date 2018-03-08 or 2019-03-07
    - they have respective cancelation Dates (2019-12-06) or 2020-02-06

Create a subset for all newlsetter subscriber who got at least 1 Zeitbrief in the last 6m (we want to get rid of the 0s):

In [ ]:
df_zb_abo = df_zeitbrief.query('received_anzahl_zeitbrief_6m >1')

In [ ]:
labels = sorted(list(set([i for i in df_zb_abo["liefer_beginn_evt"]])))

plt.figure(figsize=(95,8))
plt.title("Count of received newsletters over time")
t = sns.lineplot(data=df_zb_abo, x="liefer_beginn_evt", y="received_anzahl_zeitbrief_6m")
t.set_xticklabels(labels, rotation=60, ha="right");

In [ ]:
sns.countplot(data=df_zb_abo, x="received_anzahl_zeitbrief_6m", hue='churn')

In [ ]:
df.received_anzahl_zeitbrief_6m.value_counts(0)

In [ ]:
df.query('received_anzahl_zeitbrief_6m > 29').shape

#### We are setting max values for all opened/received columns regarding common sense & descriptives: 

In [ ]:
df_outlier = df_outlier[df_outlier.received_anzahl_zeitbrief_1w < 3]
df_outlier = df_outlier[df_outlier.received_anzahl_zeitbrief_1m < 7]
df_outlier = df_outlier[df_outlier.received_anzahl_zeitbrief_3m < 16]
df_outlier = df_outlier[df_outlier.received_anzahl_zeitbrief_6m < 29]
df_outlier = df_outlier[df_outlier.opened_anzahl_zeitbrief_1w < 3]
df_outlier = df_outlier[df_outlier.opened_anzahl_zeitbrief_1m < 7]
df_outlier = df_outlier[df_outlier.opened_anzahl_zeitbrief_3m < 16]
df_outlier = df_outlier[df_outlier.openedanzahl_zeitbrief_6m < 29]

In [ ]:
df_outlier.shape

#### Openrate / Clickrate
- you can't open more emails than you received, therefore we delete all rates with values >1 (1593 rows):

In [ ]:
df.query('openrate_zeitbrief_1w > 1').sort_values('openrate_zeitbrief_1w', ascending=False).head(1)

In [ ]:
df_outlier = df_outlier[df_outlier.openrate_zeitbrief_1w <= 1]
df_outlier = df_outlier[df_outlier.openrate_zeitbrief_1m <= 1]
df_outlier = df_outlier[df_outlier.openrate_zeitbrief_3m <= 1]
df_outlier = df_outlier[df_outlier.clickrate_zeitbrief_1w <= 1]
df_outlier = df_outlier[df_outlier.clickrate_zeitbrief_1m <= 1]
df_outlier = df_outlier[df_outlier.clickrate_zeitbrief_3m <= 1]

In [ ]:
df_outlier.shape

In [ ]:
df_outlier.describe().T

## Demographics

## After removing the Oultiers we lost 3041 rows or around 1% of our original Dataset and continue with our EDA:

In [ ]:
def add_count(plot):
    '''adds counts to bar and count plots'''
    for p in plot.patches:
        plot.annotate(format(p.get_height(), '.0f'), 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', 
                       rotation=90,
                       xytext = (0, 9), 
                       textcoords = 'offset points')

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(20, 20), sharey=True)

custom_palette_anrede = ["indianred", "navy", "limegreen", "k"]
sns.set_palette(custom_palette_anrede)

kanal_order = ["andere", "Telefonmarketing", "Eigenwerbung", 'Mailings', 'E-Mailing', 'Fremdwerbung', 'Standwerbung', 'B2B', 'Affiliate', 'SEA']
plz_order = ['6', '5','xx','0', '2', '9', '8', '7', '3', '4', '1']
zahlungweg_order = ['Bankeinzug', 'Rechnung', 'Kreditkarte', 'Paypal']
zahlungrhythmus_order = ['jährlich', 'vierteljährlich', 'halbjährlich', 'monatlich', 'zweijährlich', 'einmalige Berechnung']
                        
g = sns.countplot(data=df_zeitbrief, x='kanal', order = kanal_order, hue='anrede', ax=axes[0][0])
g.set_xticklabels(kanal_order, rotation=45, size = 10)
g.set_xlabel("Acquisition Channel",size=15)
g.set_ylabel("Count",size=12)
add_count(g)

g1 = sns.countplot(data=df_zeitbrief, x="plz_1", hue='anrede', ax=axes[0][1])
g1.set_xticklabels(plz_order, rotation=45, size = 10)
g1.set_xlabel("PLZ of current residence",size=15)
g1.set_ylabel("Count",size=12)
add_count(g1)

g2 = sns.countplot(data=df_zeitbrief, x="zahlung_weg_name", hue='anrede', ax=axes[1][0])
g2.set_xticklabels(zahlungweg_order, rotation=45, size = 10);
g2.set_xlabel("Payment Method",size=15)
g2.set_ylabel("Count",size=12)
add_count(g2)

g3 = sns.countplot(data=df_zeitbrief, x="zahlung_rhythmus_name", hue='anrede', ax=axes[1][1])
g3.set_xticklabels(zahlungrhythmus_order, rotation=45, size = 10)
g3.set_xlabel("Payment Frequency",size=15)
g3.set_ylabel("Count",size=12);
add_count(g3)

Acquisition Channel: 

    - 'others' is by far the most common acquisition channel for new subscribers, with a count of 69k subscritptions
    -  No real marketing channel other channels should be focused on:
    - 'telefonmarketing' with 27.8k and 'e-mailings' with 22..4k runners up

PLZ_ Area:

    - '2'is the most common value which corresponds to "Die Zeit" being published in Hamburg and therefore having a bigger local impact
    - most postal code areas are equaly distributed:
    - there are 17k 'xx' values which belong to subscribers outside of Germany

Payment Method:

    - the most popular payment methods are direct debit (51.7%) & invoice (45.7%) followed by minority shares of credit card & Paypal
    
Payment Frequency:

    - people tend to pay less frequent because of discounts
    - the most popular payment frequencies are yearly (62.5%), half yearly (24.4%) and quartlerly (12.7%) (financial advantages)
    - minorities pay monthly(.3%) two-yearly or one-time payment

In [ ]:
df_zeitbrief.zahlung_rhythmus_name.value_counts(1)

In [ ]:
anrede_order = ["ZEIT Digital", "Die Zeit", "Die Zeit C&W"]
met_order = ["Nein", "Ja"]

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))
f1 = sns.countplot(data=df_zeitbrief, x="objekt_name", hue="anrede", ax=axes[0]);
f1.set_title("Subscriptions by Gender", fontsize=16, loc="left");
f1.set_xticklabels(anrede_order, rotation=45, size=15);
add_count(f1)

f2 = sns.countplot(data=df_zeitbrief, x="studentenabo", hue="anrede", ax=axes[1]);
f2.set_title("Subscriptions by Gender", fontsize=16, loc="left");
f2.set_xticklabels(met_order, rotation=45, size=15);
add_count(f2)

#### Checking non-german subscribers:

In [ ]:
df_foreign = df_zeitbrief[df_zeitbrief["plz_1"] == "xx"]

In [ ]:
df_foreign.ort.value_counts()

In [ ]:
pd.pivot_table(df_zeitbrief, columns='objekt_name')

#### Churn & Churn avg

In [ ]:
df.query('churn == 1').sort_values("kuendigungs_eingangs_datum", ascending=True).shape

In [ ]:
df.churn.value_counts(1)

The dataset is slightly imbalanced with 68.9% (110,802) of the people staying subscribers and 31.1% (49,922)of the people staying in the subscription

We have a closer look at churn == 1 to check why & when people decided to quit their subscription:

In [ ]:
df_churn = df.query('churn == 1')

In [ ]:
df_churn.sort_values("kuendigungs_eingangs_datum", ascending=False).head(1)

In total we have 49922 cancellation Dates in the column "kuendigungs_eingangs_datum", which is equal to the count of churn == 1

The oldest cancellation Date is 2019-05-28

The most recent cancellation date is 2020-05-28

So we got all cancelations of the different subscription typen within a year, the Date of the data (sql) query probably was the 29th of May this year.

In [ ]:
labels = sorted(list(set([i for i in df_churn["kuendigungs_eingangs_datum"]])))

plt.figure(figsize=(55,8))
plt.title("Count of received subscription cancellations over time")
t = sns.lineplot(data=df_churn, x="kuendigungs_eingangs_datum", y="avg_churn")
t.set_xticklabels(labels, rotation=60, ha="right");

## Zeitbrief Newsletter

"Erfahren Sie am Mittwoch, was für Sie drin ist. Wir informieren Sie per Mail über die Themen und Inhalte der aktuellen gedruckten ZEIT und natürlich darüber, was Sie bei ZEIT ONLINE finden."

- is a special kind of weekly newsletter 
- is sent every wednesday and features the recently covered topics of the print issue and what's found on Zeit Online

In [ ]:
df_zeitbrief.columns

In [ ]:
df.query('received_anzahl_zeitbrief_6m >26 & nl_zeitbrief !=2').sort_values("received_anzahl_zeitbrief_6m", ascending=False).head(5)

#### Create Subset of all the 74k people who received at least 1 Zeitbrief Newsletter:

In [ ]:
df_outlier_abo = df_outlier.query('received_anzahl_zeitbrief_6m > 0')
df_outlier_abo.shape

In [ ]:
df.nl_zeitbrief.value_counts()

#### Inconsitency within the Data:
- The variable 'nl_zeitbrief' relates to subscribers to the Zeitbrief - Newsletter. 
- There 98k active subscribers but only 72k subscribers who actually received at least 1 newsletter i.t.l.6M which probably is a result of a bad Data Architecture cause of wrongly tuned APIs.

Checking the lines and creating a subset of people who did not subscribe to the newsletter but received a newsletter:

In [ ]:
df_weird = df.query('received_anzahl_zeitbrief_6m >1 & nl_zeitbrief !=2').sort_values("received_anzahl_zeitbrief_6m", ascending=False)

In [ ]:
df_weird.shape

In [ ]:
df_weird.nl_zeitbrief.value_counts()

In the subset df_weird we got 1031 subscribers who are either 0 = "not available" or 1 = "signed off" but received a newsletter in the last 6 months, the majority (1014 subscribers or 99%) have signed off the newsletter

In [ ]:
df.query("openedanzahl_zeitbrief_6m > 28").sort_values("openedanzahl_zeitbrief_6m", ascending=False).head(1)

This inconsistency cannot described by the dataset

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=4, figsize=(20, 20), sharey=False)

sns.countplot(data=df_outlier_abo, x="received_anzahl_zeitbrief_6m", hue='churn', ax=axes[0][0])
sns.countplot(data=df_outlier_abo, x="openedanzahl_zeitbrief_6m", hue='churn', ax=axes[0][1])
sns.countplot(data=df_outlier_abo, x="clicked_anzahl_zeitbrief_6m", hue='churn', ax=axes[1][0])
sns.countplot(data=df_outlier_abo, x="unsubscribed_anzahl_zeitbrief_6m", hue='churn', ax=axes[1][1])
sns.histplot(data=df_outlier_abo, x="openrate_zeitbrief_3m", hue='churn', ax=axes[2][0])
sns.histplot(data=df_outlier_abo, x="clickrate_zeitbrief_3m", hue='churn', ax=axes[2][1])
sns.countplot(data=df_outlier_abo, x="received_anzahl_zeitbrief_1w", hue='churn', ax=axes[3][0]);

#### Zeitbrief - Newsletter stats:
- there are 98533 subscribers to the newsletter (received it at least 1 time in the last 6m)
- most people who subscribe to the newsletter do not open it

#### We check the timeline of subscribers, who received more than 28 Zeitbrief Newsletters within 6 months:

In [ ]:
df_outlier.shape

In [ ]:
labels = sorted(list(set([i for i in df_outlier["liefer_beginn_evt"]])))

plt.figure(figsize=(95,8))
plt.title("Count of received newsletters over time")
t = sns.lineplot(data=df_outlier, x="liefer_beginn_evt", y="received_anzahl_zeitbrief_6m")
t.set_xticklabels(labels, rotation=60, ha="right");

In [ ]:
df.received_anzahl_1w == df.received_anzahl_bestandskunden_1w + df.received_anzahl_produktnews_1w + df.received_anzahl_hamburg_1w + df.received_anzahl_zeitbrief_1w

In [ ]:
df.head(1)